In [86]:
import numpy as np
import pandas as pd
import plotly as pt
import plotly.express as px
import yfinance as yf

In [87]:
import sys

print(sys.version)
print(pd.__version__)
print(yf.__version__)
print(pt.__version__)

3.12.11 (main, Jul  8 2025, 20:59:33) [Clang 20.1.4 ]
2.3.1
0.2.65
6.2.0


In [88]:
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]

STOCKS = [
    "^CNXREALTY",  # NIFTY REALTY
    "ANANTRAJ.NS",
    "BRIGADE.NS",
    "DLF.NS",
    "SOBHA.NS",
    "RAYMOND.NS",
    "PHOENIXLTD.NS",
    "PRESTIGE.NS",
    "LODHA.NS",
    "GODREJPROP.NS",
    "OBEROIRLTY.NS",
]


In [ ]:
def calc_annual_return(monthly_returns: pd.Series):
    return monthly_returns.add(1, fill_value=0.0).prod() - 1.0  # type: ignore


def add_monthly_contributions(df: pd.DataFrame):
    def get_monthly_contrib(monthly_series: pd.Series):
        return np.log1p(monthly_series[MONTHS]) / np.log1p(
            monthly_series["annual_returns"]
        )

    contrib = df.apply(get_monthly_contrib, axis=1).add_suffix("_contrib")
    return df.join(contrib)


def add_avg_monthly_return(df: pd.DataFrame):
    avg_monthly_returns = (
        df[MONTHS].mean(axis=0).rename("monthly_avg").to_frame().transpose()
    )
    return pd.concat([df, avg_monthly_returns])


def get_monthly_analysis(stock_data: pd.Series) -> pd.DataFrame:
    return (
        stock_data.to_frame("close")
        .resample("ME")
        .last()
        .assign(
            month=lambda df_: df_.index.strftime("%b"),  # type: ignore
            year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
            monthly_returns=lambda df_: df_["close"].pct_change(),
        )
        .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
        .pivot_table(
            index="year", columns="month", values="monthly_returns", observed=True
        )
        .assign(
            annual_returns=lambda df_: df_.loc[:, "Jan":"Dec"].agg(
                calc_annual_return, axis=1
            ),
            first_half_avg=lambda df_: df_.loc[:, "Jan":"Jun"].mean(axis=1),
            second_half_avg=lambda df_: df_.loc[:, "Jul":"Dec"].mean(axis=1),
        )
        .pipe(add_monthly_contributions)
        .pipe(add_avg_monthly_return)
    )


def format_analysis(analysis: pd.DataFrame) -> pd.DataFrame:
    return (
        analysis[
            [
                *MONTHS[:6],
                "first_half_avg",
                *MONTHS[6:],
                "second_half_avg",
                "annual_returns",
                *[f"{month}_contrib" for month in MONTHS],
            ]
        ]
        .rename(
            columns={
                "annual_returns": "Total Annual Returns",
                "first_half_avg": "Avg returns till June",
                "second_half_avg": "Avg returns after June",
                **{f"{month}_contrib": f"{month} Contribution" for month in MONTHS},
            },
            index={
                "monthly_avg": "Avg Monthly Returns",
            },
        )
        .mul(100)
        .round(2)
    )


def download_stock_data(stock: str) -> pd.Series:
    stock_data = yf.download(
        stock,
        period="max",
        multi_level_index=False,
        auto_adjust=True,
    )["Close"]  # type: ignore

    return stock_data


def convert_to_csv(stocks):
    for stock in stocks:
        stock_data = download_stock_data(stock)
        analysis = get_monthly_analysis(stock_data)
        formatted_analysis = format_analysis(analysis)
        formatted_analysis.to_csv(
            f"../data/{stock}_analysis.csv",
            index=True,
        )
        print(f"Analysis saved to {stock}_analysis.csv")


def convert_to_excel(stocks):
    with pd.ExcelWriter(
        "../data/price_action_analysis.xlsx", engine="xlsxwriter"
    ) as writer:
        for stock in stocks:
            stock_data = download_stock_data(stock)
            analysis = get_monthly_analysis(stock_data)
            formatted_analysis = format_analysis(analysis)
            formatted_analysis.to_excel(writer, index=True, sheet_name=stock[:31])
    print("Analysis saved to ../data/price_action_analysis.xlsx")


def generate_heatmap(stock: str, stock_data: pd.Series | None = None):
    if stock_data is None:
        stock_data = download_stock_data(stock)

    res = (
        stock_data.to_frame("close")
        .resample("ME")
        .last()
        .assign(
            month=lambda df_: df_.index.strftime("%b"),  # type: ignore
            year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
            monthly_returns=lambda df_: df_["close"].pct_change(),
        )
        .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
        .pivot_table(
            index="year", columns="month", values="monthly_returns", observed=True
        )
        .mul(100)
        .round(2)
    )

    fig = px.imshow(
        res,
        color_continuous_scale="RdYlGn",
        origin="upper",
        aspect="auto",
        text_auto=".2f",
        labels=dict(month="Month", year="Year", color="Return (%)"),
    )
    fig.update_xaxes(title="")
    fig.update_yaxes(title="Year")
    fig.update_layout(
        title=f"Historical Monthly Returns of the {stock}",
        coloraxis_colorbar_ticksuffix="%",
    )
    return fig


In [90]:
stock_data = download_stock_data(STOCKS[0])
# analysis = get_monthly_analysis(stock_data)
# ft = format_analysis(analysis)
# convert_to_csv(stocks)
# convert_to_excel(stocks)

[*********************100%***********************]  1 of 1 completed


In [ ]:
res = (
    get_monthly_analysis(stock_data)
    .mul(100).round(2)
    .loc["monthly_avg", MONTHS]
)

fig = px.bar(
    x=res.index,
    y=res.values,
    color=res.values
    )
# fig
res

,month,monthly_avg
0,Jan,0.20
1,Feb,-2.60
2,Mar,0.84
3,Apr,2.77
4,May,2.32
5,Jun,1.78
6,Jul,0.95
7,Aug,-3.27
8,Sep,2.78
9,Oct,2.43


In [ ]:
for stock in STOCKS:
    stock_data = download_stock_data(stock)
    fig = generate_heatmap(stock, stock_data)
    # fig.show()
    # fig.write_image(f"../images/{stock}_heatmap.png", width=1600, height=900, scale=2)


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [ ]:
res = (
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(
        month=lambda df_: df_.index.strftime("%b"),  # type: ignore
        year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
        monthly_returns=lambda df_: df_["close"].pct_change(),
    )
    .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
    .pivot_table(index="year", columns="month", values="monthly_returns", observed=True)
    .mul(100)
    .round(2)
)

fig = px.imshow(
    res,
    color_continuous_scale="RdYlGn",
    origin="upper",
    aspect="auto",
    text_auto=".2f",
    labels=dict(month="Month", year="Year", color="Return (%)"),
)
fig.update_xaxes(title="")
fig.update_yaxes(title="Year")
fig.update_layout(
    title="Historical Monthly Returns of the Nifty Realty Index",
    coloraxis_colorbar_ticksuffix="%",
)
fig

In [ ]:
# Problem Statement 1
(stock_data.to_frame("close").resample("ME").last())

,close
Date,
2010-10-31,258.341278
2010-11-30,249.592529
2010-12-31,236.677856
2011-01-31,229.178955
2011-02-28,202.331085
...,...
2025-04-30,1637.677612
2025-05-31,1744.302979
2025-06-30,1903.420654


In [ ]:
# Problem Statement 2
(
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(monthly_returns=lambda df_: df_["close"].pct_change())
)

,close,monthly_returns
Date,,
2010-10-31,258.341278,NaN
2010-11-30,249.592529,-0.033865
2010-12-31,236.677856,-0.051743
2011-01-31,229.178955,-0.031684
2011-02-28,202.331085,-0.117148
...,...,...
2025-04-30,1637.677612,0.002565
2025-05-31,1744.302979,0.065108
2025-06-30,1903.420654,0.091221


In [ ]:
# Problem Statement 2
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]
(
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(
        month=lambda df_: df_.index.strftime("%b"),  # type: ignore
        year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
        monthly_returns=lambda df_: df_["close"].pct_change(),
    )
    .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
    .pivot_table(
        index="year",
        columns="month",
        values="monthly_returns",
    )
)

/tmp/ipykernel_94660/65462311.py:26: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
year,,,,,,,,,,,,
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.033865,-0.051743
2011,-0.031684,-0.117148,0.164951,0.002357,-0.122649,0.077155,-0.007079,-0.060180,0.033244,0.011660,-0.059552,-0.047662
2012,0.200906,0.127605,-0.050863,0.013722,-0.056338,-0.048517,-0.049918,-0.010811,0.158033,0.030766,0.045779,0.009105
2013,0.010758,-0.053734,-0.065131,-0.053367,-0.042230,-0.153039,0.017068,-0.164576,0.015616,0.112359,0.025784,0.213268
2014,-0.129432,-0.069676,0.138450,-0.042854,0.160697,0.081734,0.002313,-0.081649,-0.079392,0.125229,0.076233,0.061364
2015,0.010885,0.136452,-0.116651,-0.053983,0.139963,-0.106387,-0.032892,-0.145544,0.214810,0.046129,-0.091514,0.034669
2016,-0.094378,-0.053032,0.057174,0.158592,-0.031451,0.001107,0.072613,0.037114,-0.031975,0.207085,-0.189423,0.033235
2017,0.053835,0.039197,0.139125,0.078844,-0.050818,-0.051948,0.080095,-0.013848,0.117715,0.120222,0.026224,-0.012315
2018,0.098919,-0.018249,-0.017143,0.076923,-0.073812,-0.058388,0.039279,-0.111368,-0.079959,0.043762,0.054210,-0.012211


In [ ]:
# Problem Statement 3
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]
(
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(
        month=lambda df_: df_.index.strftime("%b"),  # type: ignore
        year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
        monthly_returns=lambda df_: df_["close"].pct_change(),
    )
    .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
    .pivot_table(
        index="year",
        columns="month",
        values="monthly_returns",
    )
    .assign(
        first_half_avg=lambda df_: df_.loc[:, "Jan":"Jun"].mean(axis=1),
        second_half_avg=lambda df_: df_.loc[:, "Jul":"Dec"].mean(axis=1),
    )[[*MONTHS[:6], "first_half_avg", *MONTHS[6:], "second_half_avg"]]
)

/tmp/ipykernel_94660/3010795563.py:26: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



month,Jan,Feb,Mar,Apr,May,Jun,first_half_avg,Jul,Aug,Sep,Oct,Nov,Dec,second_half_avg
year,,,,,,,,,,,,,,
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.033865,-0.051743,-0.042804
2011,-0.031684,-0.117148,0.164951,0.002357,-0.122649,0.077155,-0.004503,-0.007079,-0.060180,0.033244,0.011660,-0.059552,-0.047662,-0.021595
2012,0.200906,0.127605,-0.050863,0.013722,-0.056338,-0.048517,0.031086,-0.049918,-0.010811,0.158033,0.030766,0.045779,0.009105,0.030492
2013,0.010758,-0.053734,-0.065131,-0.053367,-0.042230,-0.153039,-0.059457,0.017068,-0.164576,0.015616,0.112359,0.025784,0.213268,0.036587
2014,-0.129432,-0.069676,0.138450,-0.042854,0.160697,0.081734,0.023153,0.002313,-0.081649,-0.079392,0.125229,0.076233,0.061364,0.017350
2015,0.010885,0.136452,-0.116651,-0.053983,0.139963,-0.106387,0.001713,-0.032892,-0.145544,0.214810,0.046129,-0.091514,0.034669,0.004276
2016,-0.094378,-0.053032,0.057174,0.158592,-0.031451,0.001107,0.006335,0.072613,0.037114,-0.031975,0.207085,-0.189423,0.033235,0.021441
2017,0.053835,0.039197,0.139125,0.078844,-0.050818,-0.051948,0.034706,0.080095,-0.013848,0.117715,0.120222,0.026224,-0.012315,0.053016
2018,0.098919,-0.018249,-0.017143,0.076923,-0.073812,-0.058388,0.001375,0.039279,-0.111368,-0.079959,0.043762,0.054210,-0.012211,-0.011048


In [ ]:
# Problem Statement 4
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]


def calc_annual_return(monthly_returns: pd.Series):
    return monthly_returns.add(1, fill_value=0.0).prod() - 1.0  # type: ignore


(
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(
        month=lambda df_: df_.index.strftime("%b"),  # type: ignore
        year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
        monthly_returns=lambda df_: df_["close"].pct_change(),
    )
    .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
    .pivot_table(index="year", columns="month", values="monthly_returns", observed=True)
    .assign(
        annual_returns=lambda df_: df_.loc[:, "Jan":"Dec"].agg(
            calc_annual_return, axis=1
        ),
        first_half_avg=lambda df_: df_.loc[:, "Jan":"Jun"].mean(axis=1),
        second_half_avg=lambda df_: df_.loc[:, "Jul":"Dec"].mean(axis=1),
    )
    # [[*months[:6], "first_half_avg", *months[6:], "second_half_avg"]]
)

month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,annual_returns,first_half_avg,second_half_avg
year,,,,,,,,,,,,,,,
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.033865,-0.051743,-0.083856,NaN,-0.042804
2011,-0.031684,-0.117148,0.164951,0.002357,-0.122649,0.077155,-0.007079,-0.060180,0.033244,0.011660,-0.059552,-0.047662,-0.175844,-0.004503,-0.021595
2012,0.200906,0.127605,-0.050863,0.013722,-0.056338,-0.048517,-0.049918,-0.010811,0.158033,0.030766,0.045779,0.009105,0.384935,0.031086,0.030492
2013,0.010758,-0.053734,-0.065131,-0.053367,-0.042230,-0.153039,0.017068,-0.164576,0.015616,0.112359,0.025784,0.213268,-0.179721,-0.059457,0.036587
2014,-0.129432,-0.069676,0.138450,-0.042854,0.160697,0.081734,0.002313,-0.081649,-0.079392,0.125229,0.076233,0.061364,0.206888,0.023153,0.017350
2015,0.010885,0.136452,-0.116651,-0.053983,0.139963,-0.106387,-0.032892,-0.145544,0.214810,0.046129,-0.091514,0.034669,-0.034612,0.001713,0.004276
2016,-0.094378,-0.053032,0.057174,0.158592,-0.031451,0.001107,0.072613,0.037114,-0.031975,0.207085,-0.189423,0.033235,0.108788,0.006335,0.021441
2017,0.053835,0.039197,0.139125,0.078844,-0.050818,-0.051948,0.080095,-0.013848,0.117715,0.120222,0.026224,-0.012315,0.637133,0.034706,0.053016
2018,0.098919,-0.018249,-0.017143,0.076923,-0.073812,-0.058388,0.039279,-0.111368,-0.079959,0.043762,0.054210,-0.012211,-0.080256,0.001375,-0.011048


In [ ]:
# Problem Statement 5
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]




def add_monthly_contributions(df: pd.DataFrame):
    def get_montly_contrib(monthly_series: pd.Series):
        return np.log1p(monthly_series[MONTHS]) / np.log1p(
            monthly_series["annual_returns"]
        )

    contrib = df.apply(get_montly_contrib, axis=1).add_suffix("_contrib")
    globals()["contrib"] = contrib
    return df.join(contrib)


result = (
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(
        month=lambda df_: df_.index.strftime("%b"),  # type: ignore
        year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
        monthly_returns=lambda df_: df_["close"].pct_change(),
    )
    .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
    .pivot_table(index="year", columns="month", values="monthly_returns", observed=True)
    .assign(
        annual_returns=lambda df_: df_.loc[:, "Jan":"Dec"].agg(
            calc_annual_return, axis=1
        ),
        first_half_avg=lambda df_: df_.loc[:, "Jan":"Jun"].mean(axis=1),
        second_half_avg=lambda df_: df_.loc[:, "Jul":"Dec"].mean(axis=1),
    )
    .pipe(add_monthly_contributions)
    # [[*months[:6], "first_half_avg", *months[6:], "second_half_avg", "annual_returns"]]
)

result

month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,...,Mar_contrib,Apr_contrib,May_contrib,Jun_contrib,Jul_contrib,Aug_contrib,Sep_contrib,Oct_contrib,Nov_contrib,Dec_contrib
year,,,,,,,,,,,,,,,,,,,,,
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.393368,0.606632
2011,-0.031684,-0.117148,0.164951,0.002357,-0.122649,0.077155,-0.007079,-0.060180,0.033244,0.011660,...,-0.789463,-0.012171,0.676581,-0.384305,0.036733,0.320934,-0.169101,-0.059945,0.317476,0.252517
2012,0.200906,0.127605,-0.050863,0.013722,-0.056338,-0.048517,-0.049918,-0.010811,0.158033,0.030766,...,-0.160298,0.041849,-0.178065,-0.152718,-0.157244,-0.033378,0.450549,0.093051,0.137454,0.027834
2013,0.010758,-0.053734,-0.065131,-0.053367,-0.042230,-0.153039,0.017068,-0.164576,0.015616,0.112359,...,0.339955,0.276834,0.217798,0.838427,-0.085429,0.907656,-0.078215,-0.537495,-0.128500,-0.975807
2014,-0.129432,-0.069676,0.138450,-0.042854,0.160697,0.081734,0.002313,-0.081649,-0.079392,0.125229,...,0.689553,-0.232918,0.792472,0.417802,0.012288,-0.452952,-0.439898,0.627438,0.390688,0.316703
2015,0.010885,0.136452,-0.116651,-0.053983,0.139963,-0.106387,-0.032892,-0.145544,0.214810,0.046129,...,3.521177,1.575415,-3.718783,3.193224,0.949460,4.465253,-5.524050,-1.280220,2.724603,-0.967527
2016,-0.094378,-0.053032,0.057174,0.158592,-0.031451,0.001107,0.072613,0.037114,-0.031975,0.207085,...,0.538402,1.425480,-0.309453,0.010715,0.678798,0.352887,-0.314693,1.822536,-2.033641,0.316597
2017,0.053835,0.039197,0.139125,0.078844,-0.050818,-0.051948,0.080095,-0.013848,0.117715,0.120222,...,0.264248,0.153952,-0.105801,-0.108218,0.156303,-0.028288,0.225758,0.230302,0.052513,-0.025138
2018,0.098919,-0.018249,-0.017143,0.076923,-0.073812,-0.058388,0.039279,-0.111368,-0.079959,0.043762,...,0.206695,-0.885830,0.916545,0.719134,-0.460526,1.411349,0.996151,-0.511979,-0.631035,0.146863


In [ ]:
# Problem Statement 6
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]



def add_monthly_contributions(df: pd.DataFrame):
    def get_monthly_contrib(monthly_series: pd.Series):
        return np.log1p(monthly_series[MONTHS]) / np.log1p(
            monthly_series["annual_returns"]
        )

    contrib = df.apply(get_monthly_contrib, axis=1).add_suffix("_contrib")
    globals()["contrib"] = contrib
    return df.join(contrib)


def add_avg_monthly_return(df: pd.DataFrame):
    avg_monthly_returns = (
        df[MONTHS].mean(axis=0).rename("monthly_avg").to_frame().transpose()
    )
    return pd.concat([df, avg_monthly_returns])


result = (
    stock_data.to_frame("close")
    .resample("ME")
    .last()
    .assign(
        month=lambda df_: df_.index.strftime("%b"),  # type: ignore
        year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
        monthly_returns=lambda df_: df_["close"].pct_change(),
    )
    .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
    .pivot_table(index="year", columns="month", values="monthly_returns", observed=True)
    .assign(
        annual_returns=lambda df_: df_.loc[:, "Jan":"Dec"].agg(
            calc_annual_return, axis=1
        ),
        first_half_avg=lambda df_: df_.loc[:, "Jan":"Jun"].mean(axis=1),
        second_half_avg=lambda df_: df_.loc[:, "Jul":"Dec"].mean(axis=1),
    )
    .pipe(add_monthly_contributions)
    .pipe(add_avg_monthly_return)
    # [months]
    # .mean()
    # .rename("avg_return")
    # .to_frame()
    # .transpose()
    # [[*months[:6], "first_half_avg", *months[6:], "second_half_avg", "annual_returns"]]
)
result

month,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,...,Mar_contrib,Apr_contrib,May_contrib,Jun_contrib,Jul_contrib,Aug_contrib,Sep_contrib,Oct_contrib,Nov_contrib,Dec_contrib
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.393368,0.606632
2011,-0.031684,-0.117148,0.164951,0.002357,-0.122649,0.077155,-0.007079,-0.060180,0.033244,0.011660,...,-0.789463,-0.012171,0.676581,-0.384305,0.036733,0.320934,-0.169101,-0.059945,0.317476,0.252517
2012,0.200906,0.127605,-0.050863,0.013722,-0.056338,-0.048517,-0.049918,-0.010811,0.158033,0.030766,...,-0.160298,0.041849,-0.178065,-0.152718,-0.157244,-0.033378,0.450549,0.093051,0.137454,0.027834
2013,0.010758,-0.053734,-0.065131,-0.053367,-0.042230,-0.153039,0.017068,-0.164576,0.015616,0.112359,...,0.339955,0.276834,0.217798,0.838427,-0.085429,0.907656,-0.078215,-0.537495,-0.128500,-0.975807
2014,-0.129432,-0.069676,0.138450,-0.042854,0.160697,0.081734,0.002313,-0.081649,-0.079392,0.125229,...,0.689553,-0.232918,0.792472,0.417802,0.012288,-0.452952,-0.439898,0.627438,0.390688,0.316703
2015,0.010885,0.136452,-0.116651,-0.053983,0.139963,-0.106387,-0.032892,-0.145544,0.214810,0.046129,...,3.521177,1.575415,-3.718783,3.193224,0.949460,4.465253,-5.524050,-1.280220,2.724603,-0.967527
2016,-0.094378,-0.053032,0.057174,0.158592,-0.031451,0.001107,0.072613,0.037114,-0.031975,0.207085,...,0.538402,1.425480,-0.309453,0.010715,0.678798,0.352887,-0.314693,1.822536,-2.033641,0.316597
2017,0.053835,0.039197,0.139125,0.078844,-0.050818,-0.051948,0.080095,-0.013848,0.117715,0.120222,...,0.264248,0.153952,-0.105801,-0.108218,0.156303,-0.028288,0.225758,0.230302,0.052513,-0.025138
2018,0.098919,-0.018249,-0.017143,0.076923,-0.073812,-0.058388,0.039279,-0.111368,-0.079959,0.043762,...,0.206695,-0.885830,0.916545,0.719134,-0.460526,1.411349,0.996151,-0.511979,-0.631035,0.146863
2019,0.008620,0.114022,0.073988,-0.046429,0.106939,0.082005,-0.095457,0.037256,-0.106259,-0.001379,...,0.377398,-0.251361,0.537175,0.416720,-0.530447,0.193402,-0.593967,-0.007297,0.103291,0.138808


In [ ]:
# Problem Statement 5
MONTHS = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]


def calc_annual_return(monthly_returns: pd.Series):
    return monthly_returns.add(1, fill_value=0.0).prod() - 1.0  # type: ignore


def add_monthly_contributions(df: pd.DataFrame):
    def get_montly_contrib(monthly_series: pd.Series):
        return np.log1p(monthly_series[MONTHS]) / np.log1p(
            monthly_series["annual_returns"]
        )

    contrib = df.apply(get_montly_contrib, axis=1).add_suffix("_contrib")
    return df.join(contrib)


def add_avg_monthly_return(df: pd.DataFrame):
    avg_monthly_returns = (
        df[MONTHS].mean(axis=0).rename("montly_avg").to_frame().transpose()
    )
    return pd.concat([df, avg_monthly_returns])


def get_monthly_analysis(stock_data: pd.Series):
    return (
        stock_data.to_frame("close")
        .resample("ME")
        .last()
        .assign(
            month=lambda df_: df_.index.strftime("%b"),  # type: ignore
            year=lambda df_: df_.index.strftime("%Y"),  # type: ignore
            monthly_returns=lambda df_: df_["close"].pct_change(),
        )
        .astype({"month": pd.CategoricalDtype(MONTHS, ordered=True)})
        .pivot_table(
            index="year", columns="month", values="monthly_returns", observed=True
        )
        .assign(
            annual_returns=lambda df_: df_.loc[:, "Jan":"Dec"].agg(
                calc_annual_return, axis=1
            ),
            first_half_avg=lambda df_: df_.loc[:, "Jan":"Jun"].mean(axis=1),
            second_half_avg=lambda df_: df_.loc[:, "Jul":"Dec"].mean(axis=1),
        )
        .pipe(add_monthly_contributions)
        .pipe(add_avg_monthly_return)
    )